# Get shapefiles from OpenStreetMap with OSMnx

Author: [Geoff Boeing](https://geoffboeing.com/)

  - [Overview of OSMnx](http://geoffboeing.com/2016/11/osmnx-python-street-networks/)
  - [GitHub repo](https://github.com/gboeing/osmnx)
  - [Examples, demos, tutorials](https://github.com/gboeing/osmnx-examples)
  - [Documentation](https://osmnx.readthedocs.io/en/stable/)
  - [Journal article/citation](http://geoffboeing.com/publications/osmnx-complex-street-networks/)

In [ ]:
import osmnx as ox
%matplotlib inline
ox.config(log_console=True, use_cache=True)
ox.__version__

## Get the shapefile for one city, project it, display it, and save it

In [ ]:
# from some place name, create a GeoDataFrame containing the geometry of the place
city = ox.gdf_from_place('Walnut Creek, California, USA')
city

In [ ]:
# save the retrieved data as a shapefile
ox.save_gdf_shapefile(city)

In [ ]:
# project the geometry to the appropriate UTM zone (calculated automatically) then plot it
city = ox.project_gdf(city)
fig, ax = ox.plot_shape(city)

## Create a shapefile for multiple cities, project it, display it, and save it

In [ ]:
# define a list of place names
place_names = ['Berkeley, California, USA', 
               'Oakland, California, USA',
               'Piedmont, California, USA',
               'Emeryville, California, USA',
               'Alameda, Alameda County, CA, USA']

In [ ]:
# create a GeoDataFrame with rows for each place in the list
east_bay = ox.gdf_from_places(place_names, gdf_name='east_bay_cities')
east_bay

In [ ]:
# project the geometry to the appropriate UTM zone then plot it
east_bay = ox.project_gdf(east_bay)
fig, ax = ox.plot_shape(east_bay)

In [ ]:
# save the retrieved and projected data as a shapefile
ox.save_gdf_shapefile(east_bay)

## You can also construct buffered spatial geometries

In [ ]:
# pass in buffer_dist in meters
city_buffered = ox.gdf_from_place('Walnut Creek, California, USA', buffer_dist=250)
fig, ax = ox.plot_shape(city_buffered)

In [ ]:
# you can buffer multiple places in a single query
east_bay_buffered = ox.gdf_from_places(place_names, gdf_name='east_bay_cities', buffer_dist=250)
fig, ax = ox.plot_shape(east_bay_buffered, alpha=0.7)

## You can download boroughs, counties, states, or countries too

Notice the polygon geometries represent political boundaries, not physical/land boundaries.

In [ ]:
gdf = ox.gdf_from_place('Manhattan, New York, New York, USA')
gdf = ox.project_gdf(gdf)
fig, ax = ox.plot_shape(gdf)

In [ ]:
gdf = ox.gdf_from_place('Cook County, Illinois, United States')
gdf = ox.project_gdf(gdf)
fig, ax = ox.plot_shape(gdf)

In [ ]:
gdf = ox.gdf_from_place('Iowa')
gdf = ox.project_gdf(gdf)
fig, ax = ox.plot_shape(gdf)

In [ ]:
gdf = ox.gdf_from_places(['United Kingdom', 'Ireland'])
gdf = ox.project_gdf(gdf)
fig, ax = ox.plot_shape(gdf)

## Be careful to pass the right place name that OSM needs

Be specific and explicit, and sanity check the results. The function logs a warning if you get a point returned instead of a polygon. In the first example below, OSM resolves 'Melbourne, Victoria, Australia' to a single point at the center of the city. In the second example below, OSM correctly resolves 'City of Melbourne, Victoria, Australia' to the entire city and returns its polygon geometry.

In [ ]:
melbourne = ox.gdf_from_place('Melbourne, Victoria, Australia')
melbourne = ox.project_gdf(melbourne)
type(melbourne['geometry'].iloc[0])

In [ ]:
melbourne = ox.gdf_from_place('City of Melbourne, Victoria, Australia')
melbourne = ox.project_gdf(melbourne)
fig, ax = ox.plot_shape(melbourne)

## Specify you wanted a *country* if it resolves to a *city* of the same name

OSM resolves 'Mexico' to Mexico City and returns a single point at the center of the city. Instead we have a couple options:
  
  1. We can pass a dict containing a structured query to specify that we want Mexico the country instead of Mexico the city.
  2. We can also get multiple countries by passing a list of queries. These can be a mixture of strings and dicts.

In [ ]:
mexico = ox.gdf_from_place('Mexico')
mexico = ox.project_gdf(mexico)
type(mexico['geometry'].iloc[0])

In [ ]:
# instead of a string, you can pass a dict containing a structured query
mexico = ox.gdf_from_place({'country':'Mexico'})
mexico = ox.project_gdf(mexico)
fig, ax = ox.plot_shape(mexico)

In [ ]:
# you can pass multiple queries with mixed types (dicts and strings)
mx_gt_tx = ox.gdf_from_places(queries=[{'country':'Mexico'}, 'Guatemala', {'state':'Texas'}])
mx_gt_tx = ox.project_gdf(mx_gt_tx)
fig, ax = ox.plot_shape(mx_gt_tx)

## You can request a specific result number

By default, we only request 1 result from OSM. But, we can pass an optional `which_result` parameter to query OSM for *n* results and then process/return the *n*th. If you query 'France', OSM returns the country with all its overseas territories as result 2 and European France alone as result 1. Querying for 'France' returns just the first result (European France), but passing `which_result=2` instead retrieves the top 2 results from OSM and processes/returns the 2nd one (all of France's overseas territories). You could have also done this to retrieve Mexico the country instead of Mexico City above.

In [ ]:
france = ox.gdf_from_place('France', which_result=2)
france = ox.project_gdf(france)
fig, ax = ox.plot_shape(france)

In [ ]:
france = ox.gdf_from_place('France')
france = ox.project_gdf(france)
fig, ax = ox.plot_shape(france)